In [5]:
#importing stuff
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [6]:
#loading enviro using openai gym
    
environment_name = 'CartPole-v0'
env = gym.make(environment_name)    

In [7]:
#working with the environment
episodes = 5
for episode in range(1,episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action= env.action_space.sample()
        n_state,reward,done,info = env.step(action)
        score+= reward
    print('Episode:{} Score:{}'.format(episode,score))
env.close()

Episode:1 Score:57.0
Episode:2 Score:23.0
Episode:3 Score:9.0
Episode:4 Score:15.0
Episode:5 Score:45.0


In [8]:
#Understanding the environment

env.action_space

Discrete(2)

In [9]:
env.action_space.sample()

1

In [10]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [11]:
env.observation_space.sample()

array([ 3.1700671e+00, -2.3036083e+38, -2.1534350e-03,  2.0668183e+38],
      dtype=float32)

In [12]:
#Training our agent

log_path = os.path.join('Training','Logs')
log_path

'Training\\Logs'

In [13]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path)

Using cpu device


In [14]:
model.learn(total_timesteps=20000)

C:\Users\surya\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Logging to Training\Logs\PPO_8
-----------------------------
| time/              |      |
|    fps             | 246  |
|    iterations      | 1    |
|    time_elapsed    | 8    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 404         |
|    iterations           | 2           |
|    time_elapsed         | 10          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009044464 |
|    clip_fraction        | 0.0981      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00586    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.96        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0145     |
|    value_loss           | 47.7        |
-----------------------------------------
---

In [15]:
#saving and loading models
PPO_Path=os.path.join('Training','Saved Models','PPO_Model_CartPole')

In [16]:
model.save(PPO_Path)

In [17]:
del model

In [18]:
model = PPO.load(PPO_Path,env=env)

In [19]:
#evaluating the model
evaluate_policy(model,env,n_eval_episodes=10,render=True)

C:\Users\surya\AppData\Local\Programs\Python\Python310\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
Exception ignored on calling ctypes callback function: <function Win32Window._get_window_proc.<locals>.f at 0x000001C9BD4FD3F0>
Traceback (most recent call last):
  File "C:\Users\surya\AppData\Local\Programs\Python\Python310\lib\site-packages\pyglet\window\win32\__init__.py", line 747, in f
    result = event_handler(msg, wParam, lParam)
  File "C:\Users\surya\AppData\Local\Programs\Python\Python310\lib\site-packages\pyglet\window\win32\__init__.py", line 1115, in _event_syscommand
    app.event_loop.enter_blocking()
  File "C:\Users\surya\AppData\Local\Programs\Python\Python310\lib\site-packages\pyglet\app\b

(200.0, 0.0)

In [20]:
env.close()

In [23]:
#Testing our model


episodes = 5
for episode in range(1,episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action,_= model.predict(obs)
        obs,reward,done,info = env.step(action)
        score+= reward
    print('Episode:{} Score:{}'.format(episode,score))
env.close()

Episode:1 Score:[200.]
Episode:2 Score:[200.]
Episode:3 Score:[200.]
Episode:4 Score:[200.]
Episode:5 Score:[200.]


In [34]:
obs = env.reset()

In [36]:
model.predict(obs)

(array([0], dtype=int64), None)

In [37]:
action,_= model.predict(obs)

In [38]:
env.step(action)

(array([[ 0.03972845, -0.1782002 ,  0.0403034 ,  0.26381162]],
       dtype=float32),
 array([1.], dtype=float32),
 array([False]),
 [{}])

In [39]:
training_log_path = os.path.join(log_path,'PPO_8')
training_log_path

'Training\\Logs\\PPO_8'

In [40]:
!tensorboard --logdir={training_log_path}

^C


In [43]:
#adding callbacks to the training stage

from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [44]:
save_path = os.path.join('Training','Saved Models')

In [49]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env,
                            callback_on_new_best = stop_callback,
                            eval_freq=10000,
                            best_model_save_path =save_path,
                            verbose=1)

In [51]:
model = PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path)

Using cpu device


In [52]:
model.learn(total_timesteps=20000,callback=eval_callback)

Logging to Training\Logs\PPO_9
-----------------------------
| time/              |      |
|    fps             | 2298 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1365        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009621461 |
|    clip_fraction        | 0.14        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.685      |
|    explained_variance   | 0.000359    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.02        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0229     |
|    value_loss           | 47.1        |
-----------------------------------------
---

C:\Users\surya\AppData\Local\Programs\Python\Python310\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=200.00 +/- 0.00
Episode length: 200.00 +/- 0.00
------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 200          |
|    mean_reward          | 200          |
| time/                   |              |
|    total_timesteps      | 10000        |
| train/                  |              |
|    approx_kl            | 0.0059762653 |
|    clip_fraction        | 0.0519       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.611       |
|    explained_variance   | 0.206        |
|    learning_rate        | 0.0003       |
|    loss                 | 19.7         |
|    n_updates            | 40           |
|    policy_gradient_loss | -0.014       |
|    value_loss           | 71           |
------------------------------------------
New best mean reward!
Stopping training because the mean reward 200.00  is above the threshold 200


In [57]:
#Changing Policies -- specifying new neural networks

new_arch = [dict(pi=[128,128,128,128],vf=[128,128,128])]

In [58]:
model = PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path, policy_kwargs = {'net_arch':new_arch})

Using cpu device


In [59]:
model.learn(total_timesteps=20000,callback=eval_callback)

Logging to Training\Logs\PPO_11
-----------------------------
| time/              |      |
|    fps             | 2042 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1156        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014374201 |
|    clip_fraction        | 0.255       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.68       |
|    explained_variance   | -0.0023     |
|    learning_rate        | 0.0003      |
|    loss                 | 3.1         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0297     |
|    value_loss           | 27          |
-----------------------------------------
--

In [60]:
#Using a different algorithm

from stable_baselines3 import DQN
model = DQN('MlpPolicy',env,verbose=1,tensorboard_log=log_path)

Using cpu device


In [67]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\DQN_3
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.967    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 7775     |
|    time_elapsed     | 0        |
|    total_timesteps  | 70       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.923    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 9527     |
|    time_elapsed     | 0        |
|    total_timesteps  | 163      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.884    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 9717     |
|    time_elapsed     | 0        |
|    total_timesteps  | 244      |
----------------------------------
------------------------

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 108      |
|    fps              | 11213    |
|    time_elapsed     | 0        |
|    total_timesteps  | 2275     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 11278    |
|    time_elapsed     | 0        |
|    total_timesteps  | 2356     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 116      |
|    fps              | 11275    |
|    time_elapsed     | 0        |
|    total_timesteps  | 2423     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 216      |
|    fps              | 11624    |
|    time_elapsed     | 0        |
|    total_timesteps  | 4727     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 11588    |
|    time_elapsed     | 0        |
|    total_timesteps  | 4782     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 224      |
|    fps              | 11613    |
|    time_elapsed     | 0        |
|    total_timesteps  | 4892     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 324      |
|    fps              | 11787    |
|    time_elapsed     | 0        |
|    total_timesteps  | 7145     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 328      |
|    fps              | 11787    |
|    time_elapsed     | 0        |
|    total_timesteps  | 7239     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 332      |
|    fps              | 11776    |
|    time_elapsed     | 0        |
|    total_timesteps  | 7307     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 432      |
|    fps              | 11730    |
|    time_elapsed     | 0        |
|    total_timesteps  | 9572     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 436      |
|    fps              | 11719    |
|    time_elapsed     | 0        |
|    total_timesteps  | 9633     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 440      |
|    fps              | 11699    |
|    time_elapsed     | 0        |
|    total_timesteps  | 9699     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 540      |
|    fps              | 11590    |
|    time_elapsed     | 1        |
|    total_timesteps  | 11828    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 544      |
|    fps              | 11581    |
|    time_elapsed     | 1        |
|    total_timesteps  | 11900    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 548      |
|    fps              | 11591    |
|    time_elapsed     | 1        |
|    total_timesteps  | 11991    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 648      |
|    fps              | 11603    |
|    time_elapsed     | 1        |
|    total_timesteps  | 14147    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 652      |
|    fps              | 11600    |
|    time_elapsed     | 1        |
|    total_timesteps  | 14237    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 656      |
|    fps              | 11598    |
|    time_elapsed     | 1        |
|    total_timesteps  | 14338    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 756      |
|    fps              | 11580    |
|    time_elapsed     | 1        |
|    total_timesteps  | 16442    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 760      |
|    fps              | 11574    |
|    time_elapsed     | 1        |
|    total_timesteps  | 16513    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 764      |
|    fps              | 11575    |
|    time_elapsed     | 1        |
|    total_timesteps  | 16570    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 864      |
|    fps              | 11569    |
|    time_elapsed     | 1        |
|    total_timesteps  | 18681    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 868      |
|    fps              | 11580    |
|    time_elapsed     | 1        |
|    total_timesteps  | 18827    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 872      |
|    fps              | 11588    |
|    time_elapsed     | 1        |
|    total_timesteps  | 18945    |
----------------------------------
----------------------------------
| rollout/          

In [70]:
evaluate_policy(model,env,n_eval_episodes=10,render=True)

(9.8, 2.2271057451320084)

In [71]:
env.close()